In [3]:
### REINFORCEMENT LEARNING I ###
### TRAIN, SAVE, EVALUATE MODEL ###

import sys
sys.path.insert(1, "C:/Users/w_kevi02/sciebo/PhD/01 Prescriptive Maintenance/30 RQ3 - Digital Twin/PxM & PPC (R)/RL-PxM/PxM_PPC_RL/")
import gym
import stable_baselines3 as sb
from stable_baselines3.common.callbacks import EvalCallback

total_timesteps = 1e6
env = gym.make('Steel-v0')
# Callback for best model
best_callback = EvalCallback(env, best_model_save_path='./callback/model',
                        log_path='./callback/model', eval_freq=1000,
                        deterministic=True, render=False)
model = sb.PPO('MlpPolicy', env, tensorboard_log="./tensorboard/")
model.learn(total_timesteps=total_timesteps, tb_log_name='model', callback = best_callback)

Eval num_timesteps=1000, episode_reward=53.40 +/- 4.45
Episode length: 100.00 +/- 0.00
New best mean reward!
Eval num_timesteps=2000, episode_reward=49.40 +/- 2.33
Episode length: 100.00 +/- 0.00
Eval num_timesteps=3000, episode_reward=8.60 +/- 11.96
Episode length: 100.00 +/- 0.00
Eval num_timesteps=4000, episode_reward=6.40 +/- 3.20
Episode length: 100.00 +/- 0.00
Eval num_timesteps=5000, episode_reward=24.00 +/- 9.47
Episode length: 100.00 +/- 0.00
Eval num_timesteps=6000, episode_reward=18.60 +/- 12.21
Episode length: 100.00 +/- 0.00
Eval num_timesteps=7000, episode_reward=53.80 +/- 4.31
Episode length: 100.00 +/- 0.00
New best mean reward!
Eval num_timesteps=8000, episode_reward=53.40 +/- 1.96
Episode length: 100.00 +/- 0.00
Eval num_timesteps=9000, episode_reward=46.20 +/- 5.60
Episode length: 100.00 +/- 0.00
Eval num_timesteps=10000, episode_reward=48.00 +/- 4.98
Episode length: 100.00 +/- 0.00
Eval num_timesteps=11000, episode_reward=50.60 +/- 3.88
Episode length: 100.00 +/- 0.

In [4]:
### REINFORCEMENT LEARNING II ###
### TRY AND EVALUATE MY MODEL ###
import pandas as pd
import stable_baselines3 as sb
import sys
sys.path.insert(1, "C:/Users/w_kevi02/sciebo/PhD/01 Prescriptive Maintenance/30 RQ3 - Digital Twin/PxM & PPC (R)/RL-PxM/PxM_PPC_RL/")
import gym
import numpy as np
import random

env = gym.make('Steel-v0')
model = sb.PPO.load('./callback/model/best_model', env = env)
# Initilaize Reward
result_df = pd.DataFrame(np.nan, index=range(0,100), columns=['RM', 'PM', 'Quality', 'Reward'])
# Set iterations
iterations = 1
for i in range(iterations):
    # Initialize episode
    store = []
    obs = env.reset()
    done = False
    store.append([0, obs[0], env.breakdown, obs[1], 0, done])
    # Compute one episode
    while not done:
        # Get best action for state
        action, _state = model.predict(obs, deterministic=True)
        # Compute next state
        #action = random.choice([0, 1, 2, 3, 4, 5])
        obs, reward, done, info = env.step(action)
        # Store results of this episode
        store.append([action, obs[0], env.breakdown, obs[1], reward, done])
    eps_df = pd.DataFrame(store, columns=['action', 'health', 'breakdown', 'quality', 'reward', 'done'])
    # Calculate nr. of reactive maintenance interventions by counting health 'resets' and substracting PM actions
    result_df.iloc[i]['RM'] = sum(eps_df['breakdown']==True)
    # Calculate nr. of preventive maintenance interventions
    result_df.iloc[i]['PM'] = sum(eps_df['action']==0)
    # Calculate quality
    result_df.iloc[i]['Quality'] = sum(eps_df['quality'])
    # Calculate reward
    result_df.iloc[i]['Reward'] = sum(eps_df['reward'])


print("The average number of reactive maintenance interventions per episode is: ", result_df['RM'].mean())
print("The average number of preventive maintenance interventions per episode is: ", result_df['PM'].mean())
print("The average sum of non-faulty bars per episode is: ", result_df['Quality'].mean())
print("The average reward per episode is: ", result_df['Reward'].mean())


The average number of reactive maintenance interventions per episode is:  0.0
The average number of preventive maintenance interventions per episode is:  3.0
The average sum of non-faulty bars per episode is:  99.0
The average reward per episode is:  78.0
